# Neural network approach on weather dataset 

This Neural Network Approach for Truth Discovery was inspired by the model that was introuduced in 2017 by Jermaine Marshall, Arturo Argueta and Dong Wang in thier article titled "A Neural Network Approach for Truth Discovery in Social Sensing"

In [9]:
import os
import pandas as pd
import numpy as np
import os.path as op
from sklearn.model_selection import train_test_split
# KERAS LIBRARIES
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.optimizers import SGD
import time
#Loading K-folds
from sklearn.model_selection import StratifiedKFold
import sys
sys.path.insert(0, '../../models')
from PY.NN_model import get_model
import time

In [10]:
DATA_DIR = '../../../../Data'

In [11]:
# Load all the claims
claims_input = pd.read_csv(op.join(DATA_DIR, 'flights/Claims_to_work_with/claims_DL_1.csv'))
claims_input.head()

ObjectID     PropertyID PropertyValue      SourceID  \
0  AA-1623-EWR-MIA2011-12-18  DepartureGate           34    flightaware   
1  AA-1623-EWR-MIA2011-12-18    ArrivalGate          D43    flightaware   
2  AA-1623-EWR-MIA2011-12-18  DepartureGate           34   wunderground   
3  AA-1623-EWR-MIA2011-12-18    ArrivalGate          D43   wunderground   
4  AA-1623-EWR-MIA2011-12-18  DepartureGate            34       flights   

    TimeStamp  
0  2011-12-18  
1  2011-12-18  
2  2011-12-18  
3  2011-12-18  
4  2011-12-18

In [13]:
# Load the ground truth
ground_truth = pd.read_csv(op.join(DATA_DIR, 'flights/Claims_to_work_with/truths_DL_1.csv'))
ground_truth.head()

ObjectID             PropertyID        PropertyValue
0  AA-1221-MCO-ORD2011-12-01  ExpectedDepartureTime  12/01/2011 08:00 PM
1  AA-1221-MCO-ORD2011-12-01    ActualDepartureTime  12/01/2011 08:23 PM
2  AA-1221-MCO-ORD2011-12-01          DepartureGate                   17
3  AA-1221-MCO-ORD2011-12-01    ExpectedArrivalTime  12/01/2011 09:45 PM
4  AA-1221-MCO-ORD2011-12-01      ActualArrivalTime  12/01/2011 09:53 PM

In [14]:
# Build the ground_truth vector.
tc_indexes = dict()
tc_labels = np.array([], dtype=int)
sources = set()
for index, row in ground_truth.iterrows():
    tc_id = row["PropertyID"]+"|"+row["ObjectID"]+"|"+str(row["PropertyValue"])
    if tc_id not in tc_indexes:
        tc_indexes[tc_id] = tc_labels.size
        tc_labels = np.append(tc_labels, 1)
    filtered_claims = claims_input[claims_input["PropertyID"] == row["PropertyID"]]
    filtered_claims = filtered_claims[filtered_claims["ObjectID"] == row["ObjectID"]]
    
    for idx, claim in filtered_claims.iterrows():
        if str(claim["PropertyValue"]) != str(row["PropertyValue"]):
            tc_id = claim["PropertyID"]+"|"+claim["ObjectID"]+"|"+str(claim["PropertyValue"])
            if tc_id not in tc_indexes:
                tc_indexes[tc_id] = tc_labels.size
                tc_labels = np.append(tc_labels, 0)
        sources.add(claim["SourceID"])
            
    
print("Number of claims :", tc_labels.size)
print("Number of sources:", len(sources))

Number of claims : 56356
Number of sources: 34


In [15]:
# Building the sensing matrix

claims_matrix = []

for source_num, source in enumerate(sources):  # foreach claims
    claim_vector = np.zeros(len(tc_indexes), dtype=int)       # initialize the vector with the claims
    
    source_claims = claims_input[claims_input["SourceID"] == source]
    for index, row in source_claims.iterrows():
        row_claim = row["PropertyID"]+"|"+row["ObjectID"]+"|"+str(row["PropertyValue"])
        if row_claim in tc_indexes:
            claim_vector[tc_indexes[row_claim]] = 1
    claims_matrix.append(claim_vector)
    if source_num % 200 == 0:
        print("Processed source %i " % source_num)
print("Done...")
# Our claim matrix is in the form specified above.

Processed source 0 
Done...


In [16]:
claims_matrix = np.array(claims_matrix, dtype=int)
print("Sensing Matrix shape: ", claims_matrix.shape)

Sensing Matrix shape:  (34, 56356)


In [17]:
sensing_matrix_with_truth = claims_matrix.transpose()
tc_labels_shaped = tc_labels.reshape(sensing_matrix_with_truth.shape[0], 1)
claims_sources_with_truth = np.append(sensing_matrix_with_truth, tc_labels_shaped, axis=1)

In [18]:
# Defining constants...
random_seed1 = 7
# # INSTANCE'S CONSTANTS
sources_num = len(sources)
# # MODEL'S CONSTANTS
train_set_size = 0.9
test_set_size = 1-train_set_size
learning_rate = 0.025
batch_size = 20
epochs = 150  
fhl_units = sources_num
shl_units = int(sources_num/2)  # this is the default according to the paper
thl_units = shl_units

In [19]:
# # SPLIT IN TRAIN SET AND DATA SET.
train_set, test_set = train_test_split(claims_sources_with_truth, train_size=train_set_size, test_size=test_set_size, random_state=random_seed1)

In [20]:
# # EXTRACT LABELS (Yt) and SAMPLES (X) For TRAIN-SET
train_set_x = train_set[:, 0:train_set.shape[1]-1]
# print(train_set_x.shape)
train_set_y = train_set[:, train_set.shape[1]-1]
# print(train_set_y.shape)

In [21]:
# # EXTRACT LABELS (Yt) and SAMPLES (X) For TEST-SET
test_set_x = test_set[:, 0:test_set.shape[1]-1]
# print(test_set_x.shape)
test_set_y = test_set[:, test_set.shape[1]-1]
# print(test_set_y.shape)

In [22]:
X_final=np.concatenate((train_set_x,test_set_x))
y_final=np.concatenate((train_set_y,test_set_y))

In [ ]:
start=time.time()
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores1 = []
cvscores2 = []
cvscores3 = []
cvscores4 = []
num_iter=0

for train, test in kfold.split(X_final, y_final):
    num_iter=num_iter+1
    model=get_model(fhl_units, shl_units, thl_units,sources_num,random_seed1,learning_rate)
    # Fit the model
    history=model.fit(X_final[train], y_final[train], epochs=epochs, batch_size=batch_size,verbose=0)
    # evaluate the model
    scores = model.evaluate(X_final[test], y_final[test], verbose=0)
    print("------subset number", num_iter,"------")
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores1.append(scores[1] * 100)
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    cvscores2.append(scores[2] * 100)
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    cvscores3.append(scores[3] * 100)
    print("%s: %.2f%%" % (model.metrics_names[4], scores[4]*100))
    cvscores4.append(scores[4] * 100)
print("--------------------------------------------" )   
print("accuracy","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores1), np.std(cvscores1)))
print("precision","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores2), np.std(cvscores2)))
print("recall","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores3), np.std(cvscores3)))
print("f1-mesure","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores4), np.std(cvscores4)))
end=time.time()

------subset number 1 ------
accuracy: 99.33%
precision_mesure: 97.92%
recall_mesure: 99.89%
f1_mesure: 98.84%
------subset number 2 ------
accuracy: 99.11%
precision_mesure: 97.04%
recall_mesure: 100.00%
f1_mesure: 98.39%
------subset number 3 ------
accuracy: 99.25%
precision_mesure: 97.46%
recall_mesure: 100.00%
f1_mesure: 98.63%
------subset number 4 ------
accuracy: 98.95%
precision_mesure: 96.55%
recall_mesure: 100.00%
f1_mesure: 98.15%


In [ ]:
print('time of execution', end-start)